#Wine Quality

**Data Set Information:**
TBC

**Attribute Information:**

**-type:** red wine or white wine, nominal qunatity; <br>
**-fixed acidity:** grams of tataric acid per 100ml, ratio qunatity; <br>
**-volatile acidity:** grams of acetic acid per liter of wine, ratio qunatity; <br>
**-citric acid:** grams per liter, ratio qunatity; <br>
**-residual sugar:** grams per liter, ratio qunatity; <br>
**-chlorides:** grams of sodium chloride per liter, ratio qunatity; <br>
**-free sulfur dioxide:** milligrams per liter, ratio qunatity; <br>
**-total sulfur dioxide:** milligrams per liter, ratio qunatity; <br>
**-density:** grams per cubic centimeter, ratio qunatity; <br>
**-pH:** ratio qunatity; <br>
**-sulphates:** grams of potassium sulfate per liter, ratio qunatity; <br>
**-alcohol:** vol.%, ratio qunatity; <br>
**-quality:** Output variable, score between 0 and 10, subjective(?), ordinal qunantity 

Preprocessing:
Combine two .csv files (one for red wine, one for white wine) into one file with the new attribute "type".





# General Data Preparation
### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_selection import SelectPercentile, chi2, SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier

### Get the Data

In [2]:
data = pd.read_csv('wineQualityBothTypes.csv', delimiter = ';')

In [3]:
data

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,red,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,red,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,red,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,white,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,white,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,white,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,white,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


### Basic Data Information 

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6497 non-null   float64
 2   volatile acidity      6497 non-null   float64
 3   citric acid           6497 non-null   float64
 4   residual sugar        6497 non-null   float64
 5   chlorides             6497 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6497 non-null   float64
 10  sulphates             6497 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [5]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [6]:
data.describe(include = 'object')

,type
count,6497
unique,2
top,white
freq,4898


Check missing values

In [7]:
print(data.isnull().sum(axis=0))

type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


### Exploratory Data Analysis

### Quality

In [8]:
#split data into red and white again
dataRed = data.loc[data['type'] == 'red']
dataWhite = data.loc[data['type'] == 'white']

#### Histograms

fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
bins = np.arange(data['quality'].min(), data['quality'].max()+1)
plt.hist(data['quality'], bins=bins, align='left')
plt.title("Both Types")
plt.xticks(bins)
plt.xlabel('quality')

#### Split into white wine and red wine
White wine:

fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
dataWhite = data.loc[data['type'] == 'white']
bins = np.arange(dataWhite['quality'].min(), dataWhite['quality'].max()+1)
plt.hist(dataWhite['quality'], bins=bins, align='left')
plt.title("White Wine")
plt.xticks(bins)
plt.xlabel('quality')

Red wine:

fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
bins = np.arange(dataRed['quality'].min(), dataRed['quality'].max()+1)
plt.hist(dataRed['quality'], bins=bins, align='left')
plt.title("Red Wine")
plt.xticks(bins)
plt.xlabel('quality')

### Fixed Acidity vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['fixed acidity'], data['quality'])
plt.title("Both Types")
plt.xlabel('fixed acidity [g(tataric acid)/100ml]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['fixed acidity'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('fixed acidity [g(tataric acid)/100ml]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['fixed acidity'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('fixed acidity [g(tataric acid)/100ml]')
plt.ylabel('quality')
plt.show()

### Volatile Acidity vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['volatile acidity'], data['quality'])
plt.title("Both Types")
plt.xlabel('volatile acidity [g(acetic acid)/100ml]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['volatile acidity'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('volatile acidity [g(acetic acid)/100ml]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['volatile acidity'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('volatile acidity [g(acetic acid)/100ml]')
plt.ylabel('quality')
plt.show()

### Citric Acid vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['citric acid'], data['quality'])
plt.title("Both Types")
plt.xlabel('citric acid [g/l]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['citric acid'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('citric acid [g/l]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['citric acid'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('citric acid [g/l]')
plt.ylabel('quality')
plt.show()

### Residual Sugar vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['residual sugar'], data['quality'])
plt.title("Both Types")
plt.xlabel('residual sugar [g/l]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['residual sugar'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('residual sugar [g/l]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['residual sugar'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('residual sugar [g/l]')
plt.ylabel('quality')
plt.show()

### Chlorides vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['chlorides'], data['quality'])
plt.title("Both Types")
plt.xlabel('chlorides [g(sodium chloride)/l]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['chlorides'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('chlorides [g(sodium chloride)/l]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['chlorides'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('chlorides [g(sodium chloride)/l]')
plt.ylabel('quality')
plt.show()

### Free Sulfur Dioxide vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['free sulfur dioxide'], data['quality'])
plt.title("Both Types")
plt.xlabel('free sulfur dioxide [mg/l]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['free sulfur dioxide'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('free sulfur dioxide [mg/l]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['free sulfur dioxide'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('free sulfur dioxide [mg/l]')
plt.ylabel('quality')
plt.show()

### Total Sulfur Dioxide vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['total sulfur dioxide'], data['quality'])
plt.title("Both Types")
plt.xlabel('total sulfur dioxide [mg/l]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['total sulfur dioxide'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('total sulfur dioxide [mg/l]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['total sulfur dioxide'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('total sulfur dioxide [mg/l]')
plt.ylabel('quality')
plt.show()

### Density vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['density'], data['quality'])
plt.title("Both Types")
plt.xlabel('density [g/cm³]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['density'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('density [g/cm³]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['density'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('density [g/cm³]')
plt.ylabel('quality')
plt.show()

### pH value vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['pH'], data['quality'])
plt.title("Both Types")
plt.xlabel('pH')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['pH'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('pH')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['pH'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('pH')
plt.ylabel('quality')
plt.show()

### Sulphates vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['sulphates'], data['quality'])
plt.title("Both Types")
plt.xlabel('sulphates [g(potassium sulfate)/l]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['sulphates'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('sulphates [g(potassium sulfate)/l]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['sulphates'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('sulphates [g(potassium sulfate)/l]')
plt.ylabel('quality')
plt.show()

### Alcohol Content vs. Quality 

#### Both types

plt.figure(figsize=(20, 8))
#sns.jointplot(data['fixed acidity'], data['quality'], kind='reg')
plt.scatter(data['alcohol'], data['quality'])
plt.title("Both Types")
plt.xlabel('alcohol [vol%]')
plt.ylabel('quality')
plt.show()

#### Red Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataRed['alcohol'], dataRed['quality'])
plt.title("Red Wine")
plt.xlabel('alcohol [vol%]')
plt.ylabel('quality')
plt.show()

#### White Wine

plt.figure(figsize=(20, 8))
plt.scatter(dataWhite['alcohol'], dataWhite['quality'])
plt.title("White Wine")
plt.xlabel('alcohol [vol%]')
plt.ylabel('quality')
plt.show()

# 2) Preprocessing

In [9]:
#first convert the strings 'white' and 'red' to 0 and 1 respectively
mapping = {'white': 0, 'red': 1}
data = data.replace({'type': mapping})

#Then split data into red and white again with that mapping
dataWhite = dataWhite.replace({'type': mapping})
dataRed = dataRed.replace({'type': mapping})

#split the data in attributes and class
X = data.drop('quality', axis=1)
y = data['quality']

#split the data in attributes and class
XWhite = dataWhite.drop('quality', axis=1)
yWhite = dataWhite['quality']

XRed = dataRed.drop('quality', axis=1)
yRed = dataRed['quality']


##  2.1) Feature Selection 

**Choose one of the methods:**
1. SelectPercentile (chi2)
2. SelectFromModel (LinearSVC)
3. SelectFromModel (LogisticRegression)
3. SelectFromModel (ExtraTreesClassifier)

## 2.2) Data Pre-processing

**Choose one of the methods:**
1. Standardization (StandardScaler)
2. Standardization (RobustScaler)
3. MinMaxScaler
4. Normalization

## 2.3)Outliers Detection using mathematical function Z-Score


### All three features included in the following function:

In [10]:
def featureSelection(XCurr, yCurr, featureselection_method, preprocessing_method):
    if featureselection_method == 1:
        selection = SelectPercentile(chi2, percentile=5)
    elif featureselection_method == 2:
        clf = LinearSVC()
        #clf = LinearSVC(C=0.1, penalty="l1", dual=False).fit(X, y)
        selection = SelectFromModel(clf, prefit=False)
    elif featureselection_method == 3:
        clf = LogisticRegression()
        #clf = LogisticRegression(C=0.2, penalty="l2", dual=False, max_iter=200).fit(X, y)
        selection = SelectFromModel(clf, prefit=False)
    elif featureselection_method == 4:
        clf = ExtraTreesClassifier(n_estimators=50).fit(XCurr, yCurr)
        selection = SelectFromModel(clf, prefit=False)      

    clf.feature_importances_ 
    X_transformed = selection.fit_transform(XCurr, yCurr)
    columns = np.asarray(XCurr.columns.values)
    support = np.asarray(selection.get_support())
    columns_with_support = columns[support]
    print("X_transformed.shape",X_transformed.shape)

    
    #Data Pre-processing:
    if preprocessing_method == 1:
        scaler = StandardScaler()
        scaler = scaler.fit(X_transformed)
        X_scaled = scaler.transform(X_transformed)
    elif preprocessing_method == 2:
        scaler = RobustScaler()
        scaler = scaler.fit(X_transformed)
        X_scaled = scaler.transform(X_transformed)
    elif preprocessing_method == 3:
        scaler = preprocessing.MinMaxScaler()
        scaler = scaler.fit(X_transformed)
        X_scaled = scaler.transform(X_transformed)
    elif preprocessing_method == 4:
        scaler = preprocessing.Normalizer()
        scaler = scaler.fit(X_transformed)
        X_scaled = scaler.transform(X_transformed)
    print("X_scaled = ",X_scaled)
    
    
    #Outlier Detection:
    z = np.abs(stats.zscore(X_scaled))
    threshold = 20
    outliers_rows = np.where(z > threshold)
    print("\n z > threshold = ", np.where(z > threshold))
    # The first array contains the list of row numbers and second array respective column numbers
    print("\n number of outliers = ",len(set(outliers_rows[0])))
    
    #Remove Ouliers
    #X_prepared = X_scaled[(np.abs(stats.zscore(X_scaled)) < threshold).all(axis=1)]
    #X_prepared.shape
    #y = y.to_numpy()
    
    #X_train, X_test, y_train, y_test = train_test_split(X_scaled, yCurr, test_size=0.30)
    #return(X_train, X_test, y_train, y_test)
    return(X_scaled, yCurr)

In [11]:
featureselection_method = 4
preprocessing_method = 1

In [12]:
print('\n-------------------both types ------------------\n')
X, y = featureSelection(X, y, featureselection_method, preprocessing_method)

print('\n--------------------red wine -------------------\n')
XRed, yRed = featureSelection(XRed, yRed, featureselection_method, preprocessing_method)

print('\n-------------------white wine------------------\n')
XWhite, yWhite = featureSelection(XWhite, yWhite, featureselection_method, preprocessing_method)


-------------------both types ------------------

X_transformed.shape (6497, 6)
X_scaled =  [[ 2.18883292 -1.10013986 -1.44635852  1.03499282  0.19309677 -0.91546416]
 [ 3.28223494 -0.31132009 -0.86246863  0.70148631  0.99957862 -0.58006813]
 [ 2.55330026 -0.87476278 -1.09248586  0.76818761  0.79795816 -0.58006813]
 ...
 [-0.6054167  -0.02959874 -0.08394876 -0.71925142 -0.47897144 -0.91546416]
 [-0.30169391 -0.59304143 -0.10164239 -2.00325148 -1.016626    1.9354021 ]
 [-0.78765037 -0.48035289 -0.31396599 -1.7631268  -1.41986693  1.09691202]]

 z > threshold =  (array([], dtype=int64), array([], dtype=int64))

 number of outliers =  0

--------------------red wine -------------------

X_transformed.shape (1599, 5)
X_scaled =  [[ 0.96187667 -0.37913269  0.55827446 -0.57920652 -0.96024611]
 [ 1.96744245  0.62436323  0.02826077  0.1289504  -0.58477711]
 [ 1.29706527  0.22904665  0.13426351 -0.04808883 -0.58477711]
 ...
 [-0.09955388 -0.19667889 -0.53355375  0.54204194  0.54162988]
 [ 0.65

# 3) Learning Algorithms

Here we try to find the three best classifiers with their standard settings, then compare the influence of hyperparameter optimization. 

**Types:**
- Linear Classifiers: Logistic Regression, Naive Bayes Classifier
- Nearest Neighbor
- Support Vector Machines
- Decision Trees
- Random Forest
- Neural Networks

In [13]:
NN = 1

d = {}

d["Logistic Regression"] = LogisticRegression(max_iter=200)
d["Gaussian Naive Bayes"] = GaussianNB()

d["KNearest Neighbors ("+ str(NN) + ")"] = KNeighborsClassifier(n_neighbors=NN)

d["SVM rbf"] = SVC()
d["SGD Classifier"] = SGDClassifier()

d["Decision Tree"] = DecisionTreeClassifier()

d["Random Forest"] = RandomForestClassifier()

d["Multi-layer Perceptron Classifier"] = MLPClassifier(max_iter=1000)

### General Remark:
Depending on the split of test and training set sometimes the ranking of the accuracy of the 8 claassifiers changes a bit. therefore I decided to choose Random Forest, KNearest Neighbors and Decision Tree calssifiers for the hyperparameter optimization. These three seemed to be the top three choices most often with the standard parameters. Also for consistency I will compare the same three classifiers for the 3 different data sets (both wines, red wines and white wines)

## 3.1) Comparison of Classifiers with standard settings

In [78]:
def compareClassifiers(X, y, d):
    N = 2
    
    scoreList = [0]*len(d.items())
    highScoreList = [0]*len(d.items())
    nameList = ['a']*len(d.items())
    
    for i in range(0,N):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
        
        counter = 0
        for name, clf in d.items():
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            scoreList[counter] = scoreList[counter] + score
            if score > highScoreList[counter]:
                highScoreList[counter] = score
                
            nameList[counter] = name #overwrites n times but whatever 
            print("i = ", i, " counter = ", counter, "score = ", score)
            counter += 1
 
    scoreList[:] = [x / N for x in scoreList]
                             
    ranking = sorted(zip(scoreList,nameList,highScoreList))[::-1]
    #print(sorted(zip(scoreList,nameList)))
    print("\nClassifiers from best to worst:")
    for i in range(0, len(ranking)):
        print(i+1, ') {:35} averageScore: {:20}  bestScore: {:20}'.format(ranking[i][1], str(ranking[i][0]), str(ranking[i][2])))
        #print(ranking[i][1], ranking[i][0],ranking[i][2])
    return(y_test, y_train, X_test, X_train, ranking)

#old version: (-1 indent...)
y_test, y_train, X_test, X_train = train_test_split(X, y, test_size=0.20)
scoreList = []
nameList = []
counter = 0

for name, clf in d.items():
    start = time.time()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scoreList.append(accuracy_score(y_test, y_pred))
    nameList.append(name)

    end = time.time()
    print("\n--------------",name,"---------------\n")
    print("- Accuracy: %f" % score, "- Time: %0.2f" % (end - start), "seconds")
    print("\n Number of mislabeled points out of a total %d points : %d \n\n"% (X_test.shape[0], (y_test != y_pred).sum()))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))



print("###############")
ranking = sorted(zip(scoreList,nameList))[::-1]
#print(sorted(zip(scoreList,nameList)))
print("\nClassifiers from best to worst:")
for i in range(0, len(ranking)):
    print(i+1, ') {0:35} score: {1}'.format(ranking[i][1], ranking[i][0]))

#return(y_test, y_train, X_test, X_train, ranking)

### 3.1.1) Both Wines

In [79]:
y_test, y_train, X_test, X_train, rankingBoth = compareClassifiers(X, y, d)

i =  0  counter =  0 score =  0.5441025641025641
i =  0  counter =  1 score =  0.48615384615384616
i =  0  counter =  2 score =  0.6035897435897436
i =  0  counter =  3 score =  0.5579487179487179
i =  0  counter =  4 score =  0.5066666666666667
i =  0  counter =  5 score =  0.6107692307692307
i =  0  counter =  6 score =  0.6666666666666666
i =  0  counter =  7 score =  0.5820512820512821
i =  1  counter =  0 score =  0.5487179487179488
i =  1  counter =  1 score =  0.49948717948717947
i =  1  counter =  2 score =  0.6117948717948718
i =  1  counter =  3 score =  0.56
i =  1  counter =  4 score =  0.5158974358974359
i =  1  counter =  5 score =  0.58
i =  1  counter =  6 score =  0.6569230769230769
i =  1  counter =  7 score =  0.5605128205128205

Classifiers from best to worst:
1 ) Random Forest                       averageScore: 0.6617948717948718    bestScore: 0.6666666666666666  
2 ) KNearest Neighbors (1)              averageScore: 0.6076923076923078    bestScore: 0.611794871794

### 3.1.2) Red Wine only

In [ ]:
yRed_test, yRed_train, XRed_test, XRed_train, rankingRed = compareClassifiers(XRed, yRed, d)

### 3.1.3) White Wine only

In [ ]:
yWhite_test, yWhite_train, XWhite_test, XWhite_train, rankingWhite = compareClassifiers(XWhite, yWhite, d)

## 3.2) Hyperparameter Optimization

In [100]:
def hyperparameterOptimization(classifierName, param_grid, ranking, y_test, y_train, X_test, X_train):
    names = [j for i,j,k in ranking]
    index = names.index(classifierName)
    #print(ranking[index][1])

    start = time.time()
    clf_gridsearch = GridSearchCV(d.get(ranking[index][1]), param_grid, verbose=0)
    clf_gridsearch.fit(X_train, y_train)
    print(clf_gridsearch.best_params_)
    predictions = clf_gridsearch.predict(X_test)
    #print("test =",predictions)
    score_gridsearch = accuracy_score(y_test, predictions)
    end = time.time()
    print("GridSearchCV - Accuracy: %f" % score_gridsearch, "- Time: %0.2f" % (end - start), "seconds")
    print("Previous best Accuracy: ", ranking[index][2])
   
    print("\n\nAdditional statistics for prediction using best parameters:\n")
    print("Number of mislabeled points out of a total %d points : %d \n"% (X_test.shape[0], (y_test != predictions).sum()))
    print("Confusion Matrix:\n",confusion_matrix(y_test, predictions))
    print("Classsification Report: \n",classification_report(y_test, predictions))
    
    return()

### 3.2.1) RandomForest

#### Parameter Grid

In [81]:
param_grid = {'bootstrap': [True, False],
              'criterion' : ['gini', 'entropy'],
              'max_depth': [10, 50, 100, None],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'n_estimators': [100, 1000]}

#### 3.2.1.1) Both Wines

In [ ]:
hyperparameterOptimization('Random Forest', param_grid, rankingBoth, y_test, y_train, X_test, X_train)
#y_pred = clf.predict(X_test)
#score = accuracy_score(y_test, y_pred)
#print('Accuracy without optimization = ', score) #needed because train/test set split changes from previous best score!

#### 3.2.1.2) Red Wine

In [ ]:
hyperparameterOptimization('Random Forest', param_grid, rankingRed, yRed_test, yRed_train, XRed_test, XRed_train)

#### 3.2.1.3) White Wine

In [ ]:
hyperparameterOptimization('Random Forest', param_grid, rankingWhite, yWhite_test, yWhite_train, XWhite_test, XWhite_train)

### 3.2.2) KNearest Neighbors

#### Parameter Grid

In [101]:
param_grid = param_grid = {'n_neighbors': [1,2,5,10,25,50,75,100,125],
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}

#### 3.2.2.1) Both Wines

In [102]:
hyperparameterOptimization('KNearest Neighbors (1)', param_grid, rankingBoth, y_test, y_train, X_test, X_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


{'n_neighbors': 50, 'p': 1, 'weights': 'distance'}
GridSearchCV - Accuracy: 0.672308 - Time: 7.32 seconds
Previous best Accuracy:  0.6117948717948718


Additional statistics for prediction using best parameters:

Number of mislabeled points out of a total 1950 points : 639 

Confusion Matrix:
 [[  0   0   2   5   0   0   0]
 [  0   4  30  19   0   0   0]
 [  0   0 440 206   3   0   0]
 [  0   0 121 701  34   0   0]
 [  0   0   5 162 152   0   0]
 [  0   0   0  24  26  14   0]
 [  0   0   0   1   1   0   0]]
Classsification Report: 
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         7
           4       1.00      0.08      0.14        53
           5       0.74      0.68      0.71       649
           6       0.63      0.82      0.71       856
           7       0.70      0.48      0.57       319
           8       1.00      0.22      0.36        64
           9       0.00      0.00      0.00         2

    accuracy               

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


()

#### 3.2.2.2) Red Wine

In [ ]:
hyperparameterOptimization('KNearest Neighbors (1)', param_grid, rankingRed, yRed_test, yRed_train, XRed_test, XRed_train)

#### 3.2.2.3) White Wine

In [ ]:
hyperparameterOptimization('KNearest Neighbors (1)', param_grid, rankingWhite, yWhite_test, yWhite_train, XWhite_test, XWhite_train)

### 3.2.3) Decision Tree

#### Parameter Grid

In [ ]:
param_grid = {'criterion' : ['gini', 'entropy'],
              'max_depth': [10, 50, 100, None],
              'max_features' : ['auto', 'sqrt', 'log2', None],
              'splitter' : ['best', 'random'],
              'min_samples_leaf': [1, 2, 4]}

#### 3.2.3.1) Both Wines

In [ ]:
hyperparameterOptimization('Decision Tree', param_grid, rankingBoth, y_test, y_train, X_test, X_train)

#### 3.2.3.2) Red Wine

In [ ]:
hyperparameterOptimization('Decision Tree', param_grid, rankingRed, yRed_test, yRed_train, XRed_test, XRed_train)

#### 3.2.3.3) Red Wine

In [ ]:
hyperparameterOptimization('Decision Tree', param_grid, rankingWhite, yWhite_test, yWhite_train, XWhite_test, XWhite_train)